In [1]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@65dfe094

In [2]:
spark.conf.get("spark.driver.appUIAddress")

Waiting for a Spark session to start...

http://gw02.itversity.com:4053

In [3]:
def getType(o: Any) = o.getClass.getTypeName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)
val path = hdfs_home + "/dataSets/spark-guide/retail-data/by-day/*.csv"

os_name = Linux
hdfs_home = /user/kkdosapati
path = /user/kkdosapati/dataSets/spark-guide/retail-data/by-day/*.csv


/user/kkdosapati/dataSets/spark-guide/retail-data/by-day/*.csv

In [5]:
spark.conf.set("spark.sql.shuffle.partitions", "10")

In [6]:
val staticDataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(path)

staticDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [7]:
staticDataFrame.rdd.getNumPartitions

10

In [8]:
val part20 = staticDataFrame.repartition(20)
staticDataFrame.count()

part20 = [InvoiceNo: string, StockCode: string ... 6 more fields]


541909

In [9]:
staticDataFrame.rdd.getNumPartitions

10

In [10]:
part20.rdd.getNumPartitions

20

In [11]:
val part5 = staticDataFrame.coalesce(5)
staticDataFrame.count()

part5 = [InvoiceNo: string, StockCode: string ... 6 more fields]


541909

In [12]:
staticDataFrame.rdd.getNumPartitions

10

In [13]:
part5.rdd.getNumPartitions

5

In [14]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
staticDataFrame.explain()

== Physical Plan ==
*(1) FileScan csv [InvoiceNo#10,StockCode#11,Description#12,Quantity#13,InvoiceDate#14,UnitPrice#15,CustomerID#16,Country#17] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kkdosapati/dataSets/spark-guide/retail-data..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<InvoiceNo:string,StockCode:string,Description:string,Quantity:int,InvoiceDate:timestamp,Un...


In [16]:
staticDataFrame.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [17]:
import org.apache.spark.sql.functions.{date_format, col, max, min}

In [18]:
val preppedDataFrame = staticDataFrame
  .na.fill(0)
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))
  .coalesce(10).cache()

preppedDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [19]:
preppedDataFrame.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WATER TRA..

In [20]:
preppedDataFrame.rdd.getNumPartitions

10

In [21]:
preppedDataFrame.summary().show

+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|        CustomerID|    Country|day_of_week|
+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+-----------+
|  count|            541909|            541909|              540455|            541909|           541909|            541909|     541909|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|4.611113626089613|11476.974671024102|       null|       null|
| stddev|13428.417280795378| 16799.73762842766|                 NaN|218.08115785023395|96.75985306117943| 6777.908326013062|       null|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06| 

In [22]:
preppedDataFrame.describe().show

+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|        CustomerID|    Country|day_of_week|
+-------+------------------+------------------+--------------------+------------------+-----------------+------------------+-----------+-----------+
|  count|            541909|            541909|              540455|            541909|           541909|            541909|     541909|     541909|
|   mean|  559965.752026781|27623.240210938104|             20713.0|  9.55224954743324|4.611113626089614|11476.974671024102|       null|       null|
| stddev|13428.417280795376| 16799.73762842766|                 NaN|218.08115785023395|96.75985306117943| 6777.908326013062|       null|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06| 

In [23]:
preppedDataFrame.select(max("InvoiceDate")).show
preppedDataFrame.select(min("InvoiceDate")).show

+-------------------+
|   max(InvoiceDate)|
+-------------------+
|2011-12-09 12:50:00|
+-------------------+

+-------------------+
|   min(InvoiceDate)|
+-------------------+
|2010-12-01 08:26:00|
+-------------------+



In [24]:
val trainDataFrame = preppedDataFrame.where("InvoiceDate < '2011-07-01'")
val testDataFrame = preppedDataFrame.where("InvoiceDate >= '2011-07-01'")

trainDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]
testDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [25]:
println(trainDataFrame.count(), testDataFrame.count())

(245903,296006)


In [26]:
import org.apache.spark.ml.feature.StringIndexer
val indexer = new StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_index")

indexer = strIdx_e40dc9841639


strIdx_e40dc9841639

In [27]:
trainDataFrame.printSchema

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)



In [28]:
val indexerEstimator = indexer.fit(trainDataFrame).transform(trainDataFrame)
indexerEstimator.printSchema
indexerEstimator.show

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = false)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_index|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+
|   537226|    22811|SET OF 6 T-LIGHTS...|       6|2010-12-06 08:34:00|     2.95|   15987.0|United Kingdom|     Monday|              2.0|
|   537226| 

indexerEstimator = [InvoiceNo: string, StockCode: string ... 8 more fields]


[InvoiceNo: string, StockCode: string ... 8 more fields]

In [29]:
import org.apache.spark.ml.feature.OneHotEncoder
val encoder = new OneHotEncoder().setInputCol("day_of_week_index").setOutputCol("day_of_week_encoded")

encoder = oneHot_92bcac1d1270


oneHot_92bcac1d1270

In [30]:
val encoderTransformer = encoder.transform(indexerEstimator)
encoderTransformer.printSchema
encoderTransformer.show
encoderTransformer.select("day_of_week", "day_of_week_index", "day_of_week_encoded").distinct.show(false)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = false)
 |-- day_of_week_encoded: vector (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_index|day_of_week_encoded|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-----------------+-------------------+
|   537226|    22811|SET OF 6 T-LIGHTS.

encoderTransformer = [InvoiceNo: string, StockCode: string ... 9 more fields]


[InvoiceNo: string, StockCode: string ... 9 more fields]

In [31]:
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler().setInputCols(Array("UnitPrice", "Quantity", "day_of_week_encoded"))
  .setOutputCol("features")

vectorAssembler = vecAssembler_6e0e7cbd6311


vecAssembler_6e0e7cbd6311

In [32]:
val vectorAssemblerTransformer = vectorAssembler.transform(encoderTransformer)
vectorAssemblerTransformer.printSchema
vectorAssemblerTransformer.select("UnitPrice", "Quantity", "day_of_week_encoded","features", "day_of_week", "day_of_week_index").show(false)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = false)
 |-- day_of_week_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)

+---------+--------+-------------------+----------------------------+-----------+-----------------+
|UnitPrice|Quantity|day_of_week_encoded|features                    |day_of_week|day_of_week_index|
+---------+--------+-------------------+----------------------------+-----------+-----------------+
|2.95     |6       |(5,[2],[1.0])      |(7,[0,1,4],[2.95,6.0,1.0])  |Monday     |2.0              |
|2.1      |8       |(5,[2],[1.0])      |(7,[0,1,4],[2.1,8.0,1.0])   |Mond

vectorAssemblerTransformer = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [33]:
import org.apache.spark.ml.Pipeline
val transformationPipeline = new Pipeline().setStages(Array(indexer, encoder, vectorAssembler))

transformationPipeline = pipeline_5921bff7f490


pipeline_5921bff7f490

In [34]:
val fittedPipeline = transformationPipeline.fit(trainDataFrame)

fittedPipeline = pipeline_5921bff7f490


pipeline_5921bff7f490

In [35]:
val transformedTraining = fittedPipeline.transform(trainDataFrame)

transformedTraining = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [36]:
transformedTraining.cache()

[InvoiceNo: string, StockCode: string ... 10 more fields]

In [37]:
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
val kmeans = new KMeans().setK(20).setSeed(1)

kmeans = kmeans_e8cbe8c79e2c


kmeans_e8cbe8c79e2c

In [38]:
val kmModel = kmeans.fit(transformedTraining)

kmModel = kmeans_e8cbe8c79e2c


kmeans_e8cbe8c79e2c

In [39]:
val transformedTest = fittedPipeline.transform(testDataFrame)

transformedTest = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [40]:
transformedTest.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = false)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = false)
 |-- CustomerID: double (nullable = false)
 |-- Country: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- day_of_week_index: double (nullable = false)
 |-- day_of_week_encoded: vector (nullable = true)
 |-- features: vector (nullable = true)



In [41]:
val completePipeline = new Pipeline().setStages(Array(indexer, encoder, vectorAssembler, kmeans))

completePipeline = pipeline_10368edc0477


pipeline_10368edc0477

In [42]:
val fittedCompletePipeline = completePipeline.fit(trainDataFrame)

fittedCompletePipeline = pipeline_10368edc0477


pipeline_10368edc0477

In [43]:
val model = fittedCompletePipeline.stages(3).asInstanceOf[KMeansModel]

model = kmeans_e8cbe8c79e2c


kmeans_e8cbe8c79e2c

In [44]:
val trainOutput = fittedCompletePipeline.transform(trainDataFrame)
val testOutput = fittedCompletePipeline.transform(testDataFrame)

trainOutput = [InvoiceNo: string, StockCode: string ... 11 more fields]
testOutput = [InvoiceNo: string, StockCode: string ... 11 more fields]


[InvoiceNo: string, StockCode: string ... 11 more fields]

In [45]:
val trainCost = model.computeCost(trainOutput)
val kmModelTrainCost = kmModel.computeCost(transformedTraining)

trainCost = 7.441715841721837E7
kmModelTrainCost = 7.441715841721831E7


7.441715841721831E7

In [46]:
val testCost = model.computeCost(testOutput)
val kmModelTestCost = kmModel.computeCost(transformedTest)

testCost = 5.2978529789163804E8
kmModelTestCost = 5.297852978916379E8


5.297852978916379E8